In [32]:
# Imports & Settings
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from prophet import Prophet

from helpers.load_data import load_games, load_ballparks, load_teams, load_weather

pd.set_option("display.max_columns", None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
# Load Data
games_2000_2022 = load_games(start_year=2000, end_year=2022)
ballparks = load_ballparks()
teams = load_teams()
weather = load_weather()

/Users/jakemfeldman/sports_analytics/neo/helpers/load_data.py:13: DtypeWarning: Columns (19,85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  games_2010_2016 = pd.read_csv("/Users/jakemfeldman/sports_analytics/neo/helpers/games/mlb-game-logs-2010-2016.csv")


### Assumptions:
- Day games take place during that day's max temperature
- Weather only affects offensive production in outdoor environments (even retractable domes are removed due to unclear data on if dome was open or not)

TODO: offensive productivity is measured based on the following: Runs Scored, AVG, SLG, & OBP

In [ ]:
games_2000_2022["t_score"] = games_2000_2022["h_score"] + games_2000_2022["v_score"]
foo = games_2000_2022.resample(rule="D", on="date")['t_score'].mean()

In [ ]:
foo = foo.reset_index()
cols_dict = {
    "date": "ds", 
    "t_score": "y",
}
foo = foo[list(cols_dict.keys())].copy()
foo.rename(columns=cols_dict, inplace=True)


In [ ]:
foo

In [ ]:
m = Prophet()
m.fit(foo)

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


In [ ]:
fig = m.plot(forecast)

In [ ]:
fig1 = m.plot_components(forecast)

In [ ]:
# # Stadium Locations on Map

# fig = go.Figure(data=go.Scattergeo(
#         lon = stadiums['lng'],
#         lat = stadiums['lat'],
#         text = stadiums['team'],
#         mode = 'markers',
#         # marker_color = df['cnt'],
#         ))

# fig.update_layout(
#         title = 'MLB Stadium Locations (as of 2023)',
#         geo_scope='usa',
#     )
# fig.show()

In [ ]:
# foo = games_2000_2022.resample(rule="M", on="date")['t_score'].mean()
# foo = games_2000_2022.groupby(['month', 'year'])['t_score'].mean()
foo = games_2000_2022.resample(rule="D", on="date")['t_score'].mean()

In [ ]:
fig = px.box(games_2000_2022, color='month', y='t_score')
fig.show()